# C4.5算法

## np.nanmax(), np.nanmin()
- https://blog.csdn.net/weixin_36670529/article/details/116498936
- 忽略数据中的nan值计算最大，最小值

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 读取数据
data = pd.read_csv('data/titanic/train.csv')

In [10]:
data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# 删掉编号，姓名，船票
data.drop(columns=['PassengerId', 'Name', 'Ticket'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


In [8]:
# 将连续特征分组
feat_range = {}
cont_feat = ['Age', 'Fare']  # Fare为船票价格
bins = 10  # 分类点数

for feat in cont_feat:
    min_val = np.nanmin(data[feat])
    max_val = np.nanmax(data[feat])
    feat_range[feat] = np.linspace(min_val, max_val, bins).tolist()

feat_range

{'Age': [0.42,
  9.262222222222222,
  18.104444444444447,
  26.94666666666667,
  35.78888888888889,
  44.63111111111111,
  53.473333333333336,
  62.31555555555556,
  71.15777777777778,
  80.0],
 'Fare': [0.0,
  56.925466666666665,
  113.85093333333333,
  170.7764,
  227.70186666666666,
  284.62733333333335,
  341.5528,
  398.4782666666666,
  455.4037333333333,
  512.3292]}

In [9]:
# 将离散特征的值转化为整数
# Pclass为仓位等级，SibSp为登船的兄弟姐妹的数量，Parch为登船的父母子女的数量
# Cabin为船舱编号，Embarked为登船港口
cat_feat = ['Sex', 'Pclass', 'SibSp', 'Parch', 'Cabin', 'Embarked']